<a href="https://colab.research.google.com/github/MocT117/Another-one-/blob/master/e2e2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

st.set_page_config(page_title="Dashboard Proceso Importación", page_icon=":bar_chart:", layout="centered")

st.title("Dashboard Proceso Importación")
st.write("Carga tu archivo Excel para analizar tiempos logísticos:")

archivo = st.file_uploader("Sube tu archivo Excel", type=["xlsx"])

if archivo:
    df = pd.read_excel(archivo)

    # ======== DASHBOARD 1: Aduana vs Pago de Pedimento (días hábiles) ========

    st.header("Días hábiles: Fecha entrada Aduana vs Fecha pago pedimento")
    # Convertir columnas a fecha
    df['Fecha entrada Aduana'] = pd.to_datetime(df['Fecha entrada Aduana'], errors='coerce')
    df['Fecha pago pedimento'] = pd.to_datetime(df['Fecha pago pedimento'], errors='coerce')

    # Filtrar filas válidas (con ambas fechas)
    df_validas = df[df['Fecha entrada Aduana'].notna() & df['Fecha pago pedimento'].notna()].copy()

    # Calcular diferencia en días hábiles
    df_validas['Diferencia días hábiles'] = df_validas.apply(
        lambda x: np.busday_count(
            x['Fecha entrada Aduana'].date(),
            x['Fecha pago pedimento'].date()
        ) if pd.notna(x['Fecha entrada Aduana']) and pd.notna(x['Fecha pago pedimento']) else np.nan,
        axis=1
    )

    # Semáforo: On time <= 7 días hábiles
    def status_semáforo(dias):
        if pd.isna(dias):
            return 'Sin fecha'
        elif dias <= 7:
            return 'On time'
        elif dias > 7:
            return 'Delay'
        else:
            return 'Revisar'
    df_validas['Semáforo'] = df_validas['Diferencia días hábiles'].apply(status_semáforo)

    num_filas = len(df_validas)

    # Mostrar tabla
    st.subheader("Tabla Fecha entrada Aduana vs Fecha pago pedimento")
    st.dataframe(df_validas[['Fecha entrada Aduana', 'Fecha pago pedimento', 'Diferencia días hábiles', 'Semáforo']])

    # Gauge chart: promedio días hábiles
    st.subheader("Diferencia Fecha entrada Aduana vs Fecha pago pedimento (días hábiles)")

    if not df_validas['Diferencia días hábiles'].dropna().empty:
        promedio_dias = df_validas['Diferencia días hábiles'].mean()
    else:
        promedio_dias = 0

    minimo = 0
    maximo = max(14, int((df_validas['Diferencia días hábiles'].max() // 4 + 1) * 4))
    tick_vals = list(range(minimo, maximo+1, 4))

    fig = go.Figure(go.Indicator(
        mode = "gauge+number",
        value = promedio_dias,
        number = {'suffix': " días hábiles", 'font': {'size': 36}},
        title = {'text': "Fecha entrada Aduana vs Fecha pago pedimento"},
        gauge = {
            'axis': {'range': [minimo, maximo], 'tickmode': 'array', 'tickvals': tick_vals, 'tickfont': {'size': 16}},
            'steps': [
                {'range': [minimo, 7], 'color': "#ABEBC6"},    # Verde: On Time
                {'range': [7, maximo], 'color': "#F5B7B1"}     # Rojo: Delay
            ],
            'bar': {'color': "black", 'thickness': 0.25}
        }
    ))

    st.plotly_chart(fig)
    st.markdown(
        f"<div style='text-align:center; font-size:18px; margin-top:-30px; margin-bottom:10px;'><b>Promedio considerando {num_filas} diferencias</b></div>",
        unsafe_allow_html=True
    )

    # Percentiles como texto
    serie = df_validas['Diferencia días hábiles'].dropna()
    percentil_50 = serie.quantile(0.5)
    percentil_75 = serie.quantile(0.75)
    percentil_90 = serie.quantile(0.9)
    percentil_95 = serie.quantile(0.95)

    st.subheader("Resumen de percentiles")
    st.write(f"Percentil 50 (Mediana): **{percentil_50:.2f}** días hábiles")
    st.write(f"Percentil 75: **{percentil_75:.2f}** días hábiles")
    st.write(f"Percentil 90: **{percentil_90:.2f}** días hábiles")
    st.write(f"Percentil 95: **{percentil_95:.2f}** días hábiles")

    # Histograma
    st.subheader("Histograma de diferencia de días hábiles")
    fig_hist = px.histogram(df_validas, x='Diferencia días hábiles',
                            nbins=15,
                            title="Histograma de días hábiles (frecuencia por valor)")
    st.plotly_chart(fig_hist)

    # Tendencia temporal (promedio por mes)
    st.subheader("Tendencia histórica de diferencia de días hábiles")
    df_validas['AñoMes'] = df_validas['Fecha entrada Aduana'].dt.to_period('M').astype(str)
    trend = df_validas.groupby('AñoMes')['Diferencia días hábiles'].mean().reset_index()
    fig_trend = px.line(trend, x='AñoMes', y='Diferencia días hábiles', markers=True,
                        title="Promedio de días hábiles por mes")
    st.plotly_chart(fig_trend)

    # ======== DASHBOARD 2: E2E (Fecha de PO vs Fecha de remisión, días naturales) ========

    st.header("Análisis E2E: Fecha de PO vs Fecha de remisión (días naturales)")

    # Convertir columnas a fecha
    df['Fecha de PO'] = pd.to_datetime(df['Fecha de PO'], errors='coerce')
    df['Fecha de remisión'] = pd.to_datetime(df['Fecha de remisión'], errors='coerce')

    # Filtrar filas válidas
    df_e2e = df[df['Fecha de PO'].notna() & df['Fecha de remisión'].notna()].copy()

    # Calcular diferencia en días naturales
    df_e2e['Diferencia días naturales'] = (df_e2e['Fecha de remisión'] - df_e2e['Fecha de PO']).dt.days

    # Semáforo: On time <= 30 días naturales
    def status_e2e(dias):
        if pd.isna(dias):
            return 'Sin fecha'
        elif dias <= 30:
            return 'On time'
        elif dias > 30:
            return 'Delay'
        else:
            return 'Revisar'
    df_e2e['Semáforo E2E'] = df_e2e['Diferencia días naturales'].apply(status_e2e)

    num_e2e = len(df_e2e)

    # Mostrar tabla E2E
    st.subheader("Tabla E2E (Fecha de PO vs Fecha de remisión)")
    st.dataframe(df_e2e[['Fecha de PO', 'Fecha de remisión', 'Diferencia días naturales', 'Semáforo E2E']])

    # Gauge chart E2E
    st.subheader("Diferencia Fecha de PO vs Fecha de remisión (E2E, días naturales)")

    if not df_e2e['Diferencia días naturales'].dropna().empty:
        promedio_e2e = df_e2e['Diferencia días naturales'].mean()
    else:
        promedio_e2e = 0

    min_e2e = 0
    max_e2e = max(45, int((df_e2e['Diferencia días naturales'].max() // 10 + 1) * 10))
    tick_vals_e2e = list(range(min_e2e, max_e2e+1, 10))

    fig_e2e = go.Figure(go.Indicator(
        mode = "gauge+number",
        value = promedio_e2e,
        number = {'suffix': " días", 'font': {'size': 36}},
        title = {'text': "Fecha de PO vs Fecha de remisión (E2E)"},
        gauge = {
            'axis': {'range': [min_e2e, max_e2e], 'tickmode': 'array', 'tickvals': tick_vals_e2e, 'tickfont': {'size': 16}},
            'steps': [
                {'range': [min_e2e, 30], 'color': "#ABEBC6"},    # Verde: On Time
                {'range': [30, max_e2e], 'color': "#F5B7B1"}     # Rojo: Delay
            ],
            'bar': {'color': "black", 'thickness': 0.25}
        }
    ))

    st.plotly_chart(fig_e2e)
    st.markdown(
        f"<div style='text-align:center; font-size:18px; margin-top:-30px; margin-bottom:10px;'><b>Promedio considerando {num_e2e} diferencias</b></div>",
        unsafe_allow_html=True
    )

    # Percentiles como texto E2E
    serie_e2e = df_e2e['Diferencia días naturales'].dropna()
    percentil_50_e2e = serie_e2e.quantile(0.5)
    percentil_75_e2e = serie_e2e.quantile(0.75)
    percentil_90_e2e = serie_e2e.quantile(0.9)
    percentil_95_e2e = serie_e2e.quantile(0.95)

    st.subheader("Resumen de percentiles E2E")
    st.write(f"Percentil 50 (Mediana): **{percentil_50_e2e:.2f}** días")
    st.write(f"Percentil 75: **{percentil_75_e2e:.2f}** días")
    st.write(f"Percentil 90: **{percentil_90_e2e:.2f}** días")
    st.write(f"Percentil 95: **{percentil_95_e2e:.2f}** días")

    # Histograma E2E
    st.subheader("Histograma E2E de diferencia de días naturales")
    fig_hist_e2e = px.histogram(df_e2e, x='Diferencia días naturales',
                                nbins=15,
                                title="Histograma E2E (frecuencia por valor)")
    st.plotly_chart(fig_hist_e2e)

    # Tendencia temporal E2E (promedio por mes)
    st.subheader("Tendencia histórica E2E de diferencia de días naturales")
    df_e2e['AñoMes'] = df_e2e['Fecha de PO'].dt.to_period('M').astype(str)
    trend_e2e = df_e2e.groupby('AñoMes')['Diferencia días naturales'].mean().reset_index()
    fig_trend_e2e = px.line(trend_e2e, x='AñoMes', y='Diferencia días naturales', markers=True,
                            title="Promedio E2E de días naturales por mes")
    st.plotly_chart(fig_trend_e2e)

    st.info("Dashboard generado por Kai 🤖")

else:
    st.warning("Sube un archivo para ver el dashboard.")
